In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
## Task 1: Mark all launch sites on a map

In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
import numpy as np
import pandas as pd

# 0) Sanity: make sure lat/long columns exist & are numeric (adjust names if yours differ)
lat_col  = 'Lat'  if 'Lat'  in spacex_df.columns else 'Latitude'
lon_col  = 'Long' if 'Long' in spacex_df.columns else 'Longitude'
name_col = 'Launch Site' if 'Launch Site' in spacex_df.columns else ('LaunchSite' if 'LaunchSite' in spacex_df.columns else None)

spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors='coerce')
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors='coerce')

# 1) Build a marker_color column if missing
if 'marker_color' not in spacex_df.columns:
    if 'class' in spacex_df.columns:
        # IBM lab dataset usually has 'class' (1=success, 0=failure)
        spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')
    elif 'Class' in spacex_df.columns:
        spacex_df['marker_color'] = np.where(spacex_df['Class'] == 1, 'green', 'red')
    elif 'Landing_Outcome' in spacex_df.columns:
        # Fallback if you have textual outcomes
        spacex_df['marker_color'] = np.where(
            spacex_df['Landing_Outcome'].str.contains('Success', case=False, na=False),
            'green', 'red'
        )
    else:
        # Last resort: give everything a default
        spacex_df['marker_color'] = 'blue'


In [10]:
import folium
from folium.features import DivIcon

site_map = folium.Map(location=[29.7604, -95.3698], zoom_start=5, tiles="cartodbpositron")

for _, row in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    coord = [float(row[lat_col]), float(row[lon_col])]
    label = row.get(name_col, 'Launch Site') if name_col else 'Launch Site'
    color = row['marker_color']

    folium.Circle(
        coord,
        radius=1000,
        color=color,
        fill=True,
        fill_opacity=0.4
    ).add_child(folium.Popup(label, max_width=250)).add_to(site_map)

    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150, 24),
            icon_anchor=(0, 0),
            html=f"<div style='font-size:12px;color:#d35400;'><b>{label}</b></div>"
        )
    ).add_to(site_map)

site_map


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

In [12]:
# Task 2: Mark the success/failed launches for each site on the map

In [13]:
## --- Task 2: Mark success/failed launches on the map ---
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster

# 1) Detect columns (adjust here if your names differ)
name_col = "Launch Site" if "Launch Site" in spacex_df.columns else ("LaunchSite" if "LaunchSite" in spacex_df.columns else None)
lat_col  = "Lat" if "Lat" in spacex_df.columns else ("Latitude" if "Latitude" in spacex_df.columns else None)
lon_col  = "Long" if "Long" in spacex_df.columns else ("Longitude" if "Longitude" in spacex_df.columns else None)

if not all([name_col, lat_col, lon_col]):
    raise ValueError(
        "Need site name + latitude + longitude columns.\n"
        "Looked for: ('Launch Site' or 'LaunchSite'), ('Lat' or 'Latitude'), ('Long' or 'Longitude')\n"
        f"Available: {list(spacex_df.columns)}"
    )

# 2) Ensure numeric coords
spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors="coerce")
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors="coerce")

# 3) Create marker_color from success class if missing
if "marker_color" not in spacex_df.columns:
    if "class" in spacex_df.columns:
        c = pd.to_numeric(spacex_df["class"], errors="coerce")
    elif "Class" in spacex_df.columns:
        c = pd.to_numeric(spacex_df["Class"], errors="coerce")
    else:
        c = pd.Series(np.nan, index=spacex_df.index)
    spacex_df["marker_color"] = np.where(c == 1, "green", np.where(c == 0, "red", "blue"))

# 4) Create/reuse the base map centered on the first valid launch
try:
    site_map
except NameError:
    first_valid = spacex_df.dropna(subset=[lat_col, lon_col]).iloc[0]
    site_map = folium.Map(location=[float(first_valid[lat_col]), float(first_valid[lon_col])],
                          zoom_start=5, tiles="cartodbpositron")

# 5) Add a MarkerCluster and populate with colored markers
marker_cluster = MarkerCluster(name="Launch Outcomes").add_to(site_map)

for _, row in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    coord = [float(row[lat_col]), float(row[lon_col])]
    color = str(row["marker_color"])
    site  = str(row[name_col]) if name_col else "Launch"

    # Outcome text for tooltip/popup
    if "class" in spacex_df.columns:
        outcome_text = "Success" if int(row["class"]) == 1 else "Failure"
    elif "Class" in spacex_df.columns:
        outcome_text = "Success" if int(row["Class"]) == 1 else "Failure"
    else:
        outcome_text = "Outcome: N/A"

    # Use CircleMarker (no icon deps) for reliable coloring
    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.85,
        tooltip=f"{site} • {outcome_text}",
        popup=folium.Popup(f"<b>{site}</b><br>{outcome_text}", max_width=250)
    ).add_to(marker_cluster)

# 6) Optional legend
legend_html = """
<div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
     background: white; padding: 8px 10px; border: 1px solid #ccc; border-radius: 6px;">
  <div style="font-weight:600; margin-bottom:4px;">Launch Outcome</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:green;margin-right:6px;"></span>Success</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:red;margin-right:6px;"></span>Failure</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:blue;margin-right:6px;"></span>Unknown</div>
</div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))

# 7) Display map (and make sure the notebook is Trusted)
site_map


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [15]:
# --- Enhanced map: per-site success rates (color) and volume (size) ---
import pandas as pd
import numpy as np
import folium

# 1) Resolve column names (robust to typical variations)
name_col = "Launch Site" if "Launch Site" in spacex_df.columns else ("LaunchSite" if "LaunchSite" in spacex_df.columns else None)
lat_col  = "Lat" if "Lat" in spacex_df.columns else ("Latitude" if "Latitude" in spacex_df.columns else None)
lon_col  = "Long" if "Long" in spacex_df.columns else ("Longitude" if "Longitude" in spacex_df.columns else None)
class_col = "class" if "class" in spacex_df.columns else ("Class" if "Class" in spacex_df.columns else None)

if not all([name_col, lat_col, lon_col, class_col]):
    raise ValueError(
        "Need 'Launch Site'/'LaunchSite', 'Lat'/'Latitude', 'Long'/'Longitude', and 'class'/'Class'.\n"
        f"Available: {list(spacex_df.columns)}"
    )

# 2) Clean types
spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors="coerce")
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors="coerce")
spacex_df[class_col] = pd.to_numeric(spacex_df[class_col], errors="coerce").fillna(0).astype(int)

# 3) Aggregate per site
agg = (
    spacex_df.dropna(subset=[lat_col, lon_col])
    .groupby(name_col, as_index=False)
    .agg(
        total_launches=(class_col, "size"),
        successes=(class_col, lambda s: (s == 1).sum()),
        lat=(lat_col, "median"),
        lon=(lon_col, "median"),
    )
)
agg["success_rate"] = agg["successes"] / agg["total_launches"]

# 4) Helpers: map success_rate ∈ [0,1] -> color (red→green), and scale radius
def rate_to_hex(rate: float) -> str:
    rate = max(0.0, min(1.0, float(rate)))
    r = int(round(255 * (1 - rate)))
    g = int(round(255 * rate))
    b = 60
    return f"#{r:02x}{g:02x}{b:02x}"

max_total = max(1, int(agg["total_launches"].max()))
def volume_to_radius_m(n):
    # 600 m minimum; up to ~2200 m for the busiest site
    return int(600 + 1600 * (n / max_total))

# 5) Make/Reuse the map, add a feature layer for site bubbles
try:
    site_map
except NameError:
    # center on the median of all points if map doesn't exist yet
    center = [agg["lat"].median(), agg["lon"].median()]
    site_map = folium.Map(location=center, zoom_start=4, tiles="cartodbpositron")

site_layer = folium.FeatureGroup(name="Site Success Rates", show=True)
site_map.add_child(site_layer)

# 6) Add one circle per site with popup/tooltip
for _, r in agg.iterrows():
    color = rate_to_hex(r["success_rate"])
    radius = volume_to_radius_m(r["total_launches"])
    tooltip = f"{r[name_col]} • {r['success_rate']:.0%} ({int(r['successes'])}/{int(r['total_launches'])})"

    html = f"""
    <div style="font-size:13px;">
      <b>{r[name_col]}</b><br>
      Success rate: <b>{r['success_rate']:.1%}</b><br>
      Successes: <b>{int(r['successes'])}</b><br>
      Total launches: <b>{int(r['total_launches'])}</b>
    </div>
    """

    folium.Circle(
        location=[float(r["lat"]), float(r["lon"])],
        radius=radius,
        color=color,
        fill=True,
        fill_opacity=0.5,
        weight=2,
        tooltip=tooltip,
        popup=folium.Popup(html, max_width=260),
    ).add_to(site_layer)

# 7) Legend + layer control
legend_html = """
<div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
     background: white; padding: 10px 12px; border: 1px solid #ccc; border-radius: 8px; font-size: 12px;">
  <div style="font-weight:600; margin-bottom:6px;">Site Success Rate</div>
  <div style="display:flex;align-items:center;margin-bottom:4px;">
    <span style="display:inline-block;width:14px;height:14px;background:#ff003c;margin-right:6px;border:1px solid #999;"></span>0%
  </div>
  <div style="display:flex;align-items:center;margin-bottom:4px;">
    <span style="display:inline-block;width:14px;height:14px;background:#ff993c;margin-right:6px;border:1px solid #999;"></span>50%
  </div>
  <div style="display:flex;align-items:center;">
    <span style="display:inline-block;width:14px;height:14px;background:#00ff3c;margin-right:6px;border:1px solid #999;"></span>100%
  </div>
  <div style="margin-top:6px;">Circle size ∝ # Launches</div>
</div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl(collapsed=False).add_to(site_map)

# 8) Optional: show a ranked table in the notebook
display(
    agg[[name_col, "total_launches", "successes", "success_rate"]]
    .sort_values(["success_rate", "total_launches"], ascending=[False, False])
    .reset_index(drop=True)
    .rename(columns={name_col: "Launch Site", "success_rate": "Success Rate"})
    .style.format({"Success Rate": "{:.1%}"})
)

# 9) Render the map (ensure notebook is Trusted)
site_map


,Launch Site,total_launches,successes,Success Rate
0,KSC LC-39A,13,10,76.9%
1,CCAFS SLC-40,7,3,42.9%
2,VAFB SLC-4E,10,4,40.0%
3,CCAFS LC-40,26,7,26.9%


In [16]:
# --- Mark every launch: green = success (1), red = failure (0) ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 0) Make sure the cluster is on the map
try:
    marker_cluster
except NameError:
    marker_cluster = MarkerCluster(name="Launch Records")
try:
    site_map
except NameError:
    # center on the first valid launch
    first = spacex_df.dropna(subset=["Lat","Long"] if "Lat" in spacex_df.columns else ["Latitude","Longitude"]).iloc[0]
    lat_col  = "Lat"  if "Lat"  in spacex_df.columns else "Latitude"
    lon_col  = "Long" if "Long" in spacex_df.columns else "Longitude"
    site_map = folium.Map(location=[float(first[lat_col]), float(first[lon_col])], zoom_start=5, tiles="cartodbpositron")
marker_cluster.add_to(site_map)

# 1) Resolve columns (robust to common variations)
name_col = "Launch Site" if "Launch Site" in spacex_df.columns else ("LaunchSite" if "LaunchSite" in spacex_df.columns else None)
lat_col  = "Lat" if "Lat" in spacex_df.columns else ("Latitude" if "Latitude" in spacex_df.columns else None)
lon_col  = "Long" if "Long" in spacex_df.columns else ("Longitude" if "Longitude" in spacex_df.columns else None)
class_col = "class" if "class" in spacex_df.columns else ("Class" if "Class" in spacex_df.columns else None)

if not all([name_col, lat_col, lon_col, class_col]):
    raise ValueError(f"Missing required columns. Available: {list(spacex_df.columns)}")

# 2) Clean types
spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors="coerce")
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors="coerce")
spacex_df[class_col] = pd.to_numeric(spacex_df[class_col], errors="coerce").fillna(0).astype(int)

# 3) Add one marker per launch (cluster handles overlapping coords)
for _, row in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    coord = [float(row[lat_col]), float(row[lon_col])]
    is_success = int(row[class_col]) == 1
    color = "green" if is_success else "red"
    outcome_text = "Success" if is_success else "Failure"
    site = str(row[name_col])

    folium.CircleMarker(
        location=coord,
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.85,
        tooltip=f"{site} • {outcome_text}",
        popup=folium.Popup(f"<b>{site}</b><br>{outcome_text}", max_width=250)
    ).add_to(marker_cluster)

# Optional: tiny legend
legend_html = """
<div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
     background: white; padding: 8px 10px; border: 1px solid #ccc; border-radius: 6px;">
  <div style="font-weight:600; margin-bottom:4px;">Launch Outcome</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:green;margin-right:6px;"></span>Success (class=1)</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:red;margin-right:6px;"></span>Failure (class=0)</div>
</div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))

# Render (ensure the notebook is Trusted)
site_map


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

import pandas as pd
import numpy as np

# Pick the correct class column and coerce to numeric
class_col = 'class' if 'class' in spacex_df.columns else 'Class'
c = pd.to_numeric(spacex_df[class_col], errors='coerce')

# Create marker_color
spacex_df['marker_color'] = np.where(c == 1, 'green',
                              np.where(c == 0, 'red', 'blue'))

# (optional) quick check
print(spacex_df['marker_color'].value_counts(dropna=False))



marker_color
red      32
green    24
Name: count, dtype: int64


In [18]:
# Resolve column names (adjust if yours differ)
name_col = "Launch Site" if "Launch Site" in spacex_df.columns else ("LaunchSite" if "LaunchSite" in spacex_df.columns else None)
lat_col  = "Lat" if "Lat" in spacex_df.columns else ("Latitude" if "Latitude" in spacex_df.columns else None)
lon_col  = "Long" if "Long" in spacex_df.columns else ("Longitude" if "Longitude" in spacex_df.columns else None)
class_col = "class" if "class" in spacex_df.columns else ("Class" if "Class" in spacex_df.columns else None)

for _, record in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    # success/failure → color (fallback to blue if unknown)
    if class_col is not None:
        try:
            is_success = int(record[class_col]) == 1
        except Exception:
            is_success = None
    else:
        is_success = None

    color = record.get("marker_color") or ("green" if is_success else ("red" if is_success is not None else "blue"))

    # build marker
    marker = folium.Marker(
        location=[float(record[lat_col]), float(record[lon_col])],
        icon=folium.Icon(color="white", icon_color=color, icon="rocket", prefix="fa"),
        tooltip=f"{str(record[name_col]) if name_col else 'Launch'} • "
                f"{'Success' if is_success else ('Failure' if is_success is not None else 'Outcome: N/A')}",
        popup=folium.Popup(f"<b>{str(record[name_col]) if name_col else 'Launch'}</b><br>"
                           f"{'Success' if is_success else ('Failure' if is_success is not None else 'Outcome: N/A')}",
                           max_width=250),
    )

    marker_cluster.add_child(marker)

# Render the map
site_map


In [19]:
# TASK 3: Calculate the distances between a launch site to its proximities

In [20]:
# --- imports (if not already) ---
import math
import pandas as pd
import folium
from folium.plugins import MousePosition, MeasureControl

# --- ensure you have a map (center anywhere reasonable) ---
try:
    site_map
except NameError:
    site_map = folium.Map(location=[29.5597, -95.0831], zoom_start=8, tiles="cartodbpositron")

# --- add MousePosition + measure + click-to-copy popup ---
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,              # shows "Lat: <lat> Long: <lon>"
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)  # click anywhere to pop the coords

site_map


In [21]:
# === paste your picked proximity coordinates here (as [lat, lon]) ===
# Example (replace with your own from the map):
coastline_coord = [28.56300, -80.56700]
railway_coord   = [28.57210, -80.64900]
highway_coord   = [28.57200, -80.58500]
city_coord      = [28.53834, -81.37924]

# pick your launch site coordinate (from your DataFrame or earlier variable)
# Example:
site_coord = [28.5618571, -80.577366]   # LC-39A (example) -> replace with your site

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    from math import radians, sin, cos, asin, sqrt
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = phi2 - phi1
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

def _pair(v):  # quick validator/coercer
    if isinstance(v, str):
        parts = [p.strip() for p in v.replace("|", ",").split(",")]
        v = [float(parts[0]), float(parts[1])]
    return [float(v[0]), float(v[1])]

# Build dict of whatever proximities you set
proximities = {}
for lbl, var in [("coastline", "coastline_coord"),
                 ("railway", "railway_coord"),
                 ("highway", "highway_coord"),
                 ("city", "city_coord")]:
    if var in globals():
        proximities[lbl] = globals()[var]

# Draw lines + compute distances
rows = []
palette = {"coastline":"#e7298a","railway":"#d95f02","highway":"#7570b3","city":"#1b9e77"}

for label, coord in proximities.items():
    p = _pair(coord)
    folium.PolyLine([site_coord, p], color=palette.get(label,"#2b8cbe"),
                    weight=3, opacity=0.9, tooltip=f"Site ↔ {label}").add_to(site_map)
    folium.CircleMarker(p, radius=6, color=palette.get(label,"#2b8cbe"),
                        fill=True, fill_opacity=0.9, tooltip=label.title()).add_to(site_map)
    d = haversine_km(site_coord[0], site_coord[1], p[0], p[1])
    rows.append({"Proximity": label, "Distance_km": round(d, 3)})

distances_df = pd.DataFrame(rows).sort_values("Distance_km") if rows else pd.DataFrame(columns=["Proximity","Distance_km"])
display(distances_df)

site_map


,Proximity,Distance_km
0,coastline,1.020
2,highway,1.352
1,railway,7.088
3,city,78.366


In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    """Great-circle distance in km (Haversine)."""
    R = 6371.0088  # mean Earth radius (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

# --- Imports & helpers (reuse your calculate_distance) ---
import folium
from folium.plugins import MousePosition, MeasureControl

# (You already defined this)
# from math import sin, cos, sqrt, atan2, radians
# def calculate_distance(lat1, lon1, lat2, lon2): ...  # returns km

def _ensure_pair(val, name):
    """Coerce [lat, lon] (or 'lat,lon') to [float, float]."""
    if val is None:
        raise ValueError(f"{name} is None. Set it like {name} = [lat, lon].")
    if isinstance(val, str):
        parts = [p.strip() for p in val.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be [lat, lon], got: {val!r}")
        val = [parts[0], parts[1]]
    return [float(val[0]), float(val[1])]

# --- 1) Choose a launch site coordinate ---
# If you already have site_coord, this will be skipped.
try:
    site_coord
except NameError:
    # Build from launch_sites (or adapt column names below)
    # Expected columns: 'Launch Site' or 'LaunchSite', 'Lat'/'Latitude', 'Long'/'Longitude'
    name_col = "Launch Site" if "Launch Site" in launch_sites.columns else "LaunchSite"
    lat_col  = "Lat" if "Lat" in launch_sites.columns else "Latitude"
    lon_col  = "Long" if "Long" in launch_sites.columns else "Longitude"

    selected_site = launch_sites[name_col].iloc[0]   # or set to a specific site
    row = launch_sites.loc[launch_sites[name_col] == selected_site].iloc[0]
    site_coord = [float(row[lat_col]), float(row[lon_col])]
    site_name  = str(row[name_col])

# --- 2) Ensure/prepare the map with coordinate tools ---
try:
    site_map
except NameError:
    site_map = folium.Map(location=site_coord, zoom_start=12, tiles="cartodbpositron")

# Show live Lat/Lon (mouse) and a click popup for easy copy
formatter = "function(num) {return L.Util.formatNum(num, 6);};"
mouse_position = MousePosition(
    position='topleft',          # you mentioned top-left
    separator='  |  Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# Mark the launch site clearly
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35).add_to(site_map)

# --- 3) PASTE the proximity coordinates you read from the map, then re-run ---
# Examples below — replace with your actual values:
coastline_coord = [28.563000, -80.567000]
railway_coord   = [28.572100, -80.649000]
highway_coord   = [28.572000, -80.585000]
city_coord      = [28.538336, -81.379234]

# --- 4) Draw lines & compute distances (only for the coords you set) ---
proximities = {}
for label in ["coastline", "railway", "highway", "city"]:
    varname = f"{label}_coord"
    if varname in globals():
        proximities[label] = globals()[varname]

palette = {"coastline":"#e7298a", "railway":"#d95f02", "highway":"#7570b3", "city":"#1b9e77"}

if not proximities:
    print("⚠️ Paste your proximity coordinates above (as [lat, lon]) and re-run this cell.")
else:
    print("Distances from launch site:")
    for label, coord in proximities.items():
        p = _ensure_pair(coord, f"{label}_coord")

        # Draw a line and a small marker at the proximity point
        folium.PolyLine([site_coord, p], color=palette.get(label, "#2b8cbe"), weight=3, opacity=0.9,
                        tooltip=f"Launch site ↔ {label}").add_to(site_map)
        folium.CircleMarker(p, radius=6, color=palette.get(label, "#2b8cbe"),
                            fill=True, fill_opacity=0.9, tooltip=label.title()).add_to(site_map)

        # Compute and print distance (km) using your function
        km = calculate_distance(site_coord[0], site_coord[1], p[0], p[1])
        print(f"• {label.title():10s}: {km:.2f} km")

# --- 5) Render map (ensure notebook is Trusted) ---
site_map


Distances from launch site:
• Coastline : 1.02 km
• Railway   : 7.09 km
• Highway   : 1.35 km
• City      : 78.36 km


In [23]:
# === Build launch_sites (if missing) + zoom, mark proximities, and compute distances ===
import pandas as pd
import folium
from folium.plugins import MousePosition, MeasureControl
from math import radians, sin, cos, asin, sqrt

# ---------- 0) Get spacex_df ----------
try:
    spacex_df
except NameError:
    spacex_df = pd.read_csv("dataset_part_1.csv")  # fallback used in IBM lab

# ---------- 1) Resolve columns & create launch_sites ----------
def pick(df, *cands):
    for c in cands:
        if c in df.columns:
            return c
    raise KeyError(f"Missing any of columns: {cands}; available: {list(df.columns)}")

name_col = pick(spacex_df, "Launch Site", "LaunchSite")
lat_col  = pick(spacex_df, "Lat", "Latitude")
lon_col  = pick(spacex_df, "Long", "Longitude")

spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors="coerce")
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors="coerce")

launch_sites = (
    spacex_df[[name_col, lat_col, lon_col]]
    .dropna(subset=[lat_col, lon_col])
    .drop_duplicates(subset=[name_col])
    .rename(columns={name_col: "Launch Site", lat_col: "Lat", lon_col: "Long"})
    .reset_index(drop=True)
)

# ---------- 2) Choose a site (edit this if you want a specific one) ----------
try:
    selected_site
except NameError:
    selected_site = launch_sites["Launch Site"].iloc[0]

row = launch_sites.loc[launch_sites["Launch Site"] == selected_site].iloc[0]
site_name  = row["Launch Site"]
site_coord = [float(row["Lat"]), float(row["Long"])]

# ---------- 3) Map + coordinate helpers ----------
try:
    site_map
except NameError:
    site_map = folium.Map(location=site_coord, zoom_start=12, tiles="cartodbpositron")

formatter = "function(num){return L.Util.formatNum(num, 6);};"
mouse_position = MousePosition(
    position="topleft", prefix="Lat:", separator="  |  Long: ",
    num_digits=20, lat_formatter=formatter, lng_formatter=formatter
)
site_map.add_child(mouse_position)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# highlight the launch site
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35)\
    .add_child(folium.Popup(f"Launch Site: {site_name}")).add_to(site_map)

# ---------- 4) PASTE your proximity coordinates here, then re-run ----------
# Examples (replace with values you read from the map):
# coastline_coord = [28.563000, -80.567000]
# railway_coord   = [28.572100, -80.649000]
# highway_coord   = [28.572000, -80.585000]
# city_coord      = [28.538336, -81.379234]

# ---------- 5) Draw lines + compute distances ----------
palette = {"coastline":"#e7298a", "railway":"#d95f02", "highway":"#7570b3", "city":"#1b9e77"}

def _coerce_pair(val):
    if isinstance(val, str):
        parts = [p.strip() for p in val.replace("|", ",").split(",")]
        val = [parts[0], parts[1]]
    return [float(val[0]), float(val[1])]

def _haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = phi2 - phi1
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

proximities = {}
for lbl in ("coastline", "railway", "highway", "city"):
    v = globals().get(f"{lbl}_coord")
    if v is not None:
        proximities[lbl] = v

if not proximities:
    print("⚠️ No proximity coordinates set yet. Hover/click on the map (top-left shows Lat/Long), "
          "paste e.g. coastline_coord = [lat, lon] above, then re-run.")
else:
    print(f"Distances from {site_name}:")
    for label, coord in proximities.items():
        p = _coerce_pair(coord)
        folium.PolyLine([site_coord, p], color=palette.get(label, "#2b8cbe"),
                        weight=3, opacity=0.9, tooltip=f"{site_name} ↔ {label}").add_to(site_map)
        folium.CircleMarker(p, radius=6, color=palette.get(label, "#2b8cbe"),
                            fill=True, fill_opacity=0.9, tooltip=label.title()).add_to(site_map)
        d = _haversine_km(site_coord[0], site_coord[1], p[0], p[1])
        print(f"• {label.title():10s}: {d:.2f} km")

# ---------- 6) Render (ensure notebook is Trusted) ----------
site_map



Distances from CCAFS LC-40:
• Coastline : 1.01 km
• Railway   : 7.08 km
• Highway   : 1.31 km
• City      : 78.37 km


In [24]:
# --- BUILD launch_sites (from spacex_df or dataset_part_1.csv) + TASK 3 coastline distance ---

import math
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition, MeasureControl

# 0) Get a launches DataFrame
try:
    spacex_df
except NameError:
    # Fallback to the file many IBM labs created earlier
    try:
        spacex_df = pd.read_csv("dataset_part_1.csv")
    except Exception as e:
        raise ValueError(
            "No 'spacex_df' found and couldn't read 'dataset_part_1.csv'. "
            "Please run earlier steps to create 'spacex_df' or save the CSV."
        ) from e

# 1) Detect column names
name_col = "Launch Site" if "Launch Site" in spacex_df.columns else ("LaunchSite" if "LaunchSite" in spacex_df.columns else None)
lat_col  = "Lat"         if "Lat"         in spacex_df.columns else ("Latitude"    if "Latitude"    in spacex_df.columns else None)
lon_col  = "Long"        if "Long"        in spacex_df.columns else ("Longitude"   if "Longitude"   in spacex_df.columns else None)

if not all([name_col, lat_col, lon_col]):
    raise ValueError(
        "spacex_df must have site name + latitude + longitude columns.\n"
        "Looked for: ('Launch Site' or 'LaunchSite'), ('Lat' or 'Latitude'), ('Long' or 'Longitude')\n"
        f"Available: {list(spacex_df.columns)}"
    )

# 2) Ensure numeric coords
spacex_df[lat_col] = pd.to_numeric(spacex_df[lat_col], errors="coerce")
spacex_df[lon_col] = pd.to_numeric(spacex_df[lon_col], errors="coerce")

# 3) Create launch_sites as unique sites with one coord each
launch_sites = (
    spacex_df[[name_col, lat_col, lon_col]]
    .dropna(subset=[lat_col, lon_col])
    .drop_duplicates(subset=[name_col])
    .reset_index(drop=True)
    .rename(columns={name_col: "Launch Site", lat_col: "Lat", lon_col: "Long"})
)

# 4) Haversine distance (km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    from math import radians, sin, cos, asin, sqrt
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = phi2 - phi1
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

# 5) Choose a site (first by default)
selected_site = launch_sites["Launch Site"].iloc[0]
row = launch_sites.loc[launch_sites["Launch Site"] == selected_site].iloc[0]
site_name = row["Launch Site"]
site_coord = [float(row["Lat"]), float(row["Long"])]

# 6) Map with helpers
site_map = folium.Map(location=site_coord, zoom_start=11, tiles="cartodbpositron")
MousePosition(position="topright", separator=" | ", prefix="Lat | Lon:", num_digits=6).add_to(site_map)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# mark the site
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35)\
    .add_child(folium.Popup(f"Launch Site: {site_name}")).add_to(site_map)
folium.Marker(site_coord, icon=DivIcon(icon_size=(160,24), icon_anchor=(0,0),
    html=f"<div style='font-size:12px;color:#2b8cbe;'><b>{site_name}</b></div>")).add_to(site_map)

# 7) >>> FIRST run the cell, click the nearest coastline and copy lat/lon from the popup <<<
# Then set coastline_coord below and run again:

try:
    coastline_coord
except NameError:
    coastline_coord = None

# Example (replace with your clicked coords):
# coastline_coord = [28.563, -80.567]

if coastline_coord is not None:
    clat, clon = float(coastline_coord[0]), float(coastline_coord[1])
    d_km = haversine_km(site_coord[0], site_coord[1], clat, clon)

    # draw line & point
    folium.PolyLine([site_coord, [clat, clon]], weight=3).add_to(site_map)
    folium.CircleMarker([clat, clon], radius=6, color="#d95f0e", fill=True, fill_opacity=0.9,
                        popup=folium.Popup(f"Coastline<br>Distance: {d_km:.2f} km", max_width=250),
                        tooltip=f"Coastline • {d_km:.2f} km").add_to(site_map)

    # tiny summary table
    display(pd.DataFrame([{"Launch Site": site_name, "Proximity": "coastline", "Distance_km": round(d_km, 3)}]))

site_map


,Launch Site,Proximity,Distance_km
0,CCAFS LC-40,coastline,1.014


In [25]:
# Draw a line from the launch site to the coastline point
folium.PolyLine(
    locations=[site_coord, coastline_coord],
    color="#2b8cbe",   # any color
    weight=3,
    opacity=0.9,
    tooltip="Launch site ↔ Coastline"
).add_to(site_map)

# Show the map
site_map


In [26]:
print("site_coord =", site_coord, type(site_coord))
print("coastline_coord =", coastline_coord, type(coastline_coord))


site_coord = [28.56230197, -80.57735648] <class 'list'>
coastline_coord = [28.563, -80.567] <class 'list'>


In [27]:
# example — replace with YOUR values from the map
coastline_coord = [28.563000, -80.567000]   # [lat, lon]


In [28]:
def _ensure_pair(val, name):
    if val is None:
        raise ValueError(f"{name} is None. Set it like {name} = [lat, lon].")
    # convert strings like "28.563, -80.567" if someone pasted that
    if isinstance(val, str):
        parts = [p.strip() for p in val.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be a [lat, lon] pair, got: {val!r}")
        val = [float(parts[0]), float(parts[1])]
    # convert pandas/np types to list
    if hasattr(val, "__len__") and len(val) == 2:
        return [float(val[0]), float(val[1])]
    raise ValueError(f"{name} must be a [lat, lon] pair, got: {val!r}")

site_point = _ensure_pair(site_coord, "site_coord")
coast_point = _ensure_pair(coastline_coord, "coastline_coord")

# draw the line + endpoints
folium.PolyLine(
    locations=[site_point, coast_point],
    color="#2b8cbe", weight=3, opacity=0.9,
    tooltip="Launch site ↔ Coastline"
).add_to(site_map)

folium.CircleMarker(site_point, radius=5, color="#2b8cbe", fill=True).add_to(site_map)
folium.CircleMarker(coast_point, radius=5, color="#d95f0e", fill=True).add_to(site_map)

site_map


In [29]:
# Make sure these exist and are [lat, lon]
# site_coord = [launch_site_lat, launch_site_lon]
# coastline_coord = [coast_lat, coast_lon]

if not (isinstance(site_coord, (list, tuple)) and len(site_coord) == 2):
    raise ValueError("site_coord must be [lat, lon]")
if not (isinstance(coastline_coord, (list, tuple)) and len(coastline_coord) == 2):
    raise ValueError("coastline_coord must be [lat, lon]")

coordinates = [
    [float(site_coord[0]), float(site_coord[1])],
    [float(coastline_coord[0]), float(coastline_coord[1])]
]

lines = folium.PolyLine(locations=coordinates, color="#2b8cbe", weight=3, opacity=0.9)
site_map.add_child(lines)

# (optional) show endpoints
folium.CircleMarker(coordinates[0], radius=5, color="#2b8cbe", fill=True).add_to(site_map)
folium.CircleMarker(coordinates[1], radius=5, color="#d95f0e", fill=True).add_to(site_map)

site_map  # display


In [30]:
# === Task 3 extension: draw lines to nearest city / railway / highway, etc. ===
import math
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition, MeasureControl

# --- Helpers ---
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    from math import radians, sin, cos, asin, sqrt
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = phi2 - phi1
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

def _ensure_pair(val, name):
    if val is None:
        raise ValueError(f"{name} is None. Set it like {name} = [lat, lon].")
    if isinstance(val, str):
        parts = [p.strip() for p in val.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be a [lat, lon] pair, got: {val!r}")
        val = [float(parts[0]), float(parts[1])]
    if hasattr(val, "__len__") and len(val) == 2:
        return [float(val[0]), float(val[1])]
    raise ValueError(f"{name} must be a [lat, lon] pair, got: {val!r}")

# --- Reuse or derive launch site & map ---
try:
    site_coord  # [lat, lon] of selected launch site
    site_name
except NameError:
    # Build from launch_sites or spacex_df/dataset_part_1.csv
    try:
        launch_sites
    except NameError:
        try:
            spacex_df  # table of launches
        except NameError:
            spacex_df = pd.read_csv("dataset_part_1.csv")
        # detect columns and create launch_sites
        name_col = "Launch Site" if "Launch Site" in spacex_df.columns else "LaunchSite"
        lat_col  = "Lat" if "Lat" in spacex_df.columns else "Latitude"
        lon_col  = "Long" if "Long" in spacex_df.columns else "Longitude"
        launch_sites = (
            spacex_df[[name_col, lat_col, lon_col]]
            .dropna(subset=[lat_col, lon_col])
            .drop_duplicates(subset=[name_col])
            .rename(columns={name_col: "Launch Site", lat_col: "Lat", lon_col: "Long"})
            .reset_index(drop=True)
        )
    selected_site = launch_sites["Launch Site"].iloc[0]
    row = launch_sites.loc[launch_sites["Launch Site"] == selected_site].iloc[0]
    site_name = row["Launch Site"]
    site_coord = [float(row["Lat"]), float(row["Long"])]

try:
    site_map
except NameError:
    site_map = folium.Map(location=site_coord, zoom_start=11, tiles="cartodbpositron")

# Add coordinate tools (safe to add multiple times)
MousePosition(position="topright", separator=" | ", prefix="Lat | Lon:", num_digits=6).add_to(site_map)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# Mark the launch site
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35)\
    .add_child(folium.Popup(f"Launch Site: {site_name}")).add_to(site_map)
folium.Marker(site_coord, icon=DivIcon(icon_size=(160,24), icon_anchor=(0,0),
    html=f"<div style='font-size:12px;color:#2b8cbe;'><b>{site_name}</b></div>")).add_to(site_map)

# --- STEP 1: Run this cell and use the map readout (top-right) or click to get coords. ---
# --- STEP 2: Paste the coordinates you found below, then re-run. ---

# Examples (UNCOMMENT & REPLACE with your picked points):
# city_coord    = [28.538336, -81.379234]
# railway_coord = [28.572100, -80.649000]
# highway_coord = [28.572000, -80.585000]
# coastline_coord = [28.563000, -80.567000]

# --- Define proximities you’ve captured ---
proximities = {
    # "city": city_coord,
    # "railway": railway_coord,
    # "highway": highway_coord,
    # "coastline": coastline_coord,
}

# --- Draw lines & compute distances ---
color_by_label = {"city": "#1b9e77", "railway": "#d95f02", "highway": "#7570b3", "coastline": "#e7298a"}
dist_rows = []

for label, coord in list(proximities.items()):
    try:
        p = _ensure_pair(coord, f"{label}_coord")
    except Exception as e:
        print(f"Skipping '{label}': {e}")
        continue

    # line
    folium.PolyLine([site_coord, p], color=color_by_label.get(label, "#2b8cbe"), weight=3, opacity=0.9,
                    tooltip=f"{site_name} ↔ {label.title()}").add_to(site_map)
    # endpoint marker
    folium.CircleMarker(p, radius=6, color=color_by_label.get(label, "#2b8cbe"),
                        fill=True, fill_opacity=0.9,
                        popup=folium.Popup(f"{label.title()}", max_width=250),
                        tooltip=f"{label.title()}").add_to(site_map)

    km = haversine_km(site_coord[0], site_coord[1], p[0], p[1])
    dist_rows.append({"Launch Site": site_name, "Proximity": label, "Distance_km": round(km, 3)})

# Distance summary
distances_df = pd.DataFrame(dist_rows).sort_values("Distance_km") if dist_rows else pd.DataFrame(
    columns=["Launch Site", "Proximity", "Distance_km"]
)
display(distances_df)  # if display() isn't available, use: print(distances_df)

site_map


,Launch Site,Proximity,Distance_km


In [31]:
# ---------- Task 3: Proximity lines + auto-answers ----------
import math
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition, MeasureControl

# --- Helpers ---
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    from math import radians, sin, cos, asin, sqrt
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = phi2 - phi1
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

def _ensure_pair(val, name):
    if val is None:
        return None
    if isinstance(val, str):
        parts = [p.strip() for p in val.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be [lat, lon], got: {val!r}")
        val = [float(parts[0]), float(parts[1])]
    if hasattr(val, "__len__") and len(val) == 2:
        return [float(val[0]), float(val[1])]
    raise ValueError(f"{name} must be [lat, lon], got: {val!r}")

# --- Get a launch site and map (reuse if you already have them) ---
try:
    site_coord  # [lat, lon]
    site_name
except NameError:
    # Build from previous lab artifacts
    try:
        launch_sites
    except NameError:
        try:
            spacex_df
        except NameError:
            spacex_df = pd.read_csv("dataset_part_1.csv")  # fallback used in IBM lab
        # detect columns and create launch_sites
        name_col = "Launch Site" if "Launch Site" in spacex_df.columns else "LaunchSite"
        lat_col  = "Lat" if "Lat" in spacex_df.columns else "Latitude"
        lon_col  = "Long" if "Long" in spacex_df.columns else "Longitude"
        launch_sites = (
            spacex_df[[name_col, lat_col, lon_col]]
            .dropna(subset=[lat_col, lon_col])
            .drop_duplicates(subset=[name_col])
            .rename(columns={name_col: "Launch Site", lat_col: "Lat", lon_col: "Long"})
            .reset_index(drop=True)
        )
    sel = launch_sites["Launch Site"].iloc[0]
    row = launch_sites.loc[launch_sites["Launch Site"] == sel].iloc[0]
    site_name = row["Launch Site"]
    site_coord = [float(row["Lat"]), float(row["Long"])]

try:
    site_map
except NameError:
    site_map = folium.Map(location=site_coord, zoom_start=11, tiles="cartodbpositron")

# Tools to read coords & measure
MousePosition(position="topright", separator=" | ", prefix="Lat | Lon:", num_digits=6).add_to(site_map)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# Mark the site
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35)\
    .add_child(folium.Popup(f"Launch Site: {site_name}")).add_to(site_map)
folium.Marker(site_coord, icon=DivIcon(icon_size=(160,24), icon_anchor=(0,0),
    html=f"<div style='font-size:12px;color:#2b8cbe;'><b>{site_name}</b></div>")).add_to(site_map)

# --- STEP 1: Run once, read coordinates from the map (top-right or click). ---
# --- STEP 2: Paste them here and run again. ---

# Examples (UNCOMMENT and replace with your values):
# city_coord     = [28.538336, -81.379234]
# railway_coord  = [28.572100, -80.649000]
# highway_coord  = [28.572000, -80.585000]
# coastline_coord = [28.563000, -80.567000]

# Build proximities dict from whatever you set (it’s OK if some are missing)
proximities = {}
for label in ["city", "railway", "highway", "coastline"]:
    varname = f"{label}_coord"
    if varname in globals():
        proximities[label] = globals()[varname]

# --- Draw lines and compute distances ---
color_by_label = {"city": "#1b9e77", "railway": "#d95f02", "highway": "#7570b3", "coastline": "#e7298a"}
dist_rows = []

if not proximities:
    print("⚠️ No proximity coordinates set yet. Use the map to read Lat|Lon, paste into the *_coord variables above, then re-run.")
else:
    for label, coord in proximities.items():
        p = _ensure_pair(coord, f"{label}_coord")
        if p is None:
            continue
        # line + endpoint
        folium.PolyLine([site_coord, p], color=color_by_label.get(label, "#2b8cbe"),
                        weight=3, opacity=0.9, tooltip=f"{site_name} ↔ {label.title()}").add_to(site_map)
        folium.CircleMarker(p, radius=6, color=color_by_label.get(label, "#2b8cbe"),
                            fill=True, fill_opacity=0.9,
                            popup=folium.Popup(f"{label.title()}", max_width=250),
                            tooltip=f"{label.title()}").add_to(site_map)
        # distance
        km = haversine_km(site_coord[0], site_coord[1], p[0], p[1])
        dist_rows.append({"Launch Site": site_name, "Proximity": label, "Distance_km": round(km, 3)})

    distances_df = pd.DataFrame(dist_rows).sort_values("Distance_km")
    try:
        display(distances_df)
    except Exception:
        print(distances_df)

    # --- Auto-answers (tweak thresholds if your instructor specifies) ---
    # Heuristics:
    # close to railway/highway/coastline: <= 5 km
    # keeps distance from cities: >= 25 km
    d = {r["Proximity"]: r["Distance_km"] for r in dist_rows}

    def yn(cond):
        return "Yes" if cond else "No"

    rail_ans = yn("railway" in d and d["railway"] <= 5)
    high_ans = yn("highway" in d and d["highway"] <= 5)
    coast_ans = yn("coastline" in d and d["coastline"] <= 5)
    city_ans = yn("city" in d and d["city"] >= 25)

    print("\nAnswers:")
    if "railway" in d:  print(f"- Are launch sites in close proximity to railways? {rail_ans} (≈ {d['railway']} km)")
    if "highway" in d:  print(f"- Are launch sites in close proximity to highways? {high_ans} (≈ {d['highway']} km)")
    if "coastline" in d:print(f"- Are launch sites in close proximity to coastline? {coast_ans} (≈ {d['coastline']} km)")
    if "city" in d:     print(f"- Do launch sites keep certain distance away from cities? {city_ans} (≈ {d['city']} km)")

    # Brief findings/explanations
    findings = []
    if "railway" in d:
        findings.append(f"Railway: ~{d['railway']} km — {'close' if d['railway']<=5 else 'not especially close'} for heavy logistics access.")
    if "highway" in d:
        findings.append(f"Highway: ~{d['highway']} km — {'close' if d['highway']<=5 else 'some distance'} balancing access and safety.")
    if "coastline" in d:
        findings.append(f"Coastline: ~{d['coastline']} km — {'near coast, beneficial for downrange safety and recovery' if d['coastline']<=5 else 'farther inland than typical coastal pads'}.")
    if "city" in d:
        findings.append(f"City: ~{d['city']} km — {'keeps a safe buffer from dense population' if d['city']>=25 else 'relatively near urban area; check local safety corridors'}.")
    if findings:
        print("\nFindings:")
        for f in findings:
            print("• " + f)

# Map should be last line to render
site_map


,Launch Site,Proximity,Distance_km
3,CCAFS LC-40,coastline,1.014
2,CCAFS LC-40,highway,1.312
1,CCAFS LC-40,railway,7.081
0,CCAFS LC-40,city,78.367



Answers:
- Are launch sites in close proximity to railways? No (≈ 7.081 km)
- Are launch sites in close proximity to highways? Yes (≈ 1.312 km)
- Are launch sites in close proximity to coastline? Yes (≈ 1.014 km)
- Do launch sites keep certain distance away from cities? Yes (≈ 78.367 km)

Findings:
• Railway: ~7.081 km — not especially close for heavy logistics access.
• Highway: ~1.312 km — close balancing access and safety.
• Coastline: ~1.014 km — near coast, beneficial for downrange safety and recovery.
• City: ~78.367 km — keeps a safe buffer from dense population.


In [32]:
import folium
from folium.features import DivIcon

# ==== 1) Paste the coastline coordinates you read from MousePosition ====
# Example shown — replace with YOUR numbers:
coastline_lat, coastline_lon = 28.56367, -80.57163  # <-- change these

# ==== 2) Provide the launch site coordinate ====
# If you already have site_coord = [lat, lon], we reuse it; otherwise set explicitly:
try:
    launch_site_lat, launch_site_lon = float(site_coord[0]), float(site_coord[1])
except Exception:
    # Fallback — set your launch site lat/lon here if site_coord doesn't exist:
    # launch_site_lat, launch_site_lon = 28.5618571, -80.577366
    raise ValueError("Set site_coord = [lat, lon] or provide launch_site_lat/launch_site_lon above.")

# ==== 3) Ensure distance function exists (Haversine, km) ====
try:
    calculate_distance
except NameError:
    from math import sin, cos, sqrt, atan2, radians
    def calculate_distance(lat1, lon1, lat2, lon2):
        R = 6371.0088  # km
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        return 2 * R * atan2(sqrt(a), sqrt(1 - a))

# ==== 4) Compute distance ====
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# ==== 5) Draw the line from launch site to coastline ====
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="#2b8cbe", weight=3, opacity=0.9,
    tooltip="Launch site ↔ Coastline"
).add_to(site_map)

# ==== 6) Mark the selected coastline point ====
folium.CircleMarker(
    [coastline_lat, coastline_lon],
    radius=6, color="#d95f0e", fill=True, fill_opacity=0.9,
    tooltip="Closest coastline", popup=folium.Popup("Closest coastline", max_width=200)
).add_to(site_map)

# ==== 7) Add a distance label using DivIcon (placed at the midpoint for readability) ====
mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2
distance_marker = folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(120, 20),
        icon_anchor=(0, 0),
        html=f"<div style='font-size:12px; color:#d35400; background:white; padding:2px 4px; border-radius:3px;'>{distance_coastline:0.2f} KM</div>"
    )
)
distance_marker.add_to(site_map)

# Show the map (make sure the notebook is Trusted)
site_map


In [33]:
import folium

# --- Get launch site coordinate ---
try:
    site = [float(site_coord[0]), float(site_coord[1])]
except Exception:
    site = [float(launch_site_lat), float(launch_site_lon)]

# --- Get coastline coordinate (picked via MousePosition) ---
try:
    coast = [float(coastline_coord[0]), float(coastline_coord[1])]
except Exception:
    coast = [float(coastline_lat), float(coastline_lon)]

# --- Build the coordinate pair and draw the line ---
coordinates = [site, coast]

lines = folium.PolyLine(
    locations=coordinates,
    weight=3,
    color="#2b8cbe",
    opacity=0.9,
    tooltip="Launch site ↔ Coastline"
)
site_map.add_child(lines)

# (optional) mark the endpoints so they’re obvious
folium.CircleMarker(site, radius=5, color="#2b8cbe", fill=True).add_to(site_map)
folium.CircleMarker(coast, radius=5, color="#d95f0e", fill=True).add_to(site_map)

# render map
site_map


In [34]:
# --- Draw lines from launch site to nearest city / railway / highway / coastline ---
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition, MeasureControl
from math import radians, sin, cos, asin, sqrt

# 0) Ensure a map + a launch site coordinate exist
try:
    site_map
except NameError:
    site_map = folium.Map(location=[29.5597, -95.0831], zoom_start=8, tiles="cartodbpositron")

try:
    site_coord  # [lat, lon] of your chosen launch site
except NameError:
    # If you don't already have site_coord, set it explicitly:
    site_coord = [28.5618571, -80.577366]  # example: LC-39A — replace with your site
    raise ValueError("Set site_coord = [lat, lon] before running this cell.")

# 1) Add helpers to read coordinates from the map
formatter = "function(num){return L.Util.formatNum(num, 6);};"
MousePosition(position='topleft', prefix='Lat:', separator='  |  Long: ',
              num_digits=20, lat_formatter=formatter, lng_formatter=formatter).add_to(site_map)
site_map.add_child(MeasureControl(position="topleft", primary_length_unit="kilometers"))
folium.LatLngPopup().add_to(site_map)

# 2)
city_coord      = [28.538336, -81.379234]
railway_coord   = [28.572100, -80.649000]
highway_coord   = [28.572000, -80.585000]
coastline_coord = [28.563000, -80.567000]

# 3) Distance function (Haversine, km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    p1, p2 = radians(lat1), radians(lat2)
    dphi   = p2 - p1
    dlmb   = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(p1)*cos(p2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

def _pair(v, name):
    if v is None:
        return None
    if isinstance(v, str):
        parts = [p.strip() for p in v.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be [lat, lon], got {v!r}")
        v = [parts[0], parts[1]]
    return [float(v[0]), float(v[1])]

# 4) Collect whatever you set
proximities = {
    "city":      _pair(globals().get("city_coord"), "city_coord"),
    "railway":   _pair(globals().get("railway_coord"), "railway_coord"),
    "highway":   _pair(globals().get("highway_coord"), "highway_coord"),
    "coastline": _pair(globals().get("coastline_coord"), "coastline_coord"),
}
# drop Nones
proximities = {k:v for k,v in proximities.items() if v is not None}

# 5) Draw line + endpoint marker + distance label
palette = {"city":"#1b9e77", "railway":"#d95f02", "highway":"#7570b3", "coastline":"#e7298a"}

# Mark the launch site for reference
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35).add_to(site_map)

for label, p in proximities.items():
    # line
    folium.PolyLine([site_coord, p], color=palette.get(label, "#2b8cbe"),
                    weight=3, opacity=0.9, tooltip=f"Launch site ↔ {label}").add_to(site_map)
    # endpoint marker
    folium.CircleMarker(p, radius=6, color=palette.get(label, "#2b8cbe"),
                        fill=True, fill_opacity=0.9, tooltip=label.title()).add_to(site_map)
    # distance label at midpoint
    d_km = haversine_km(site_coord[0], site_coord[1], p[0], p[1])
    mid = [(site_coord[0]+p[0])/2, (site_coord[1]+p[1])/2]
    folium.Marker(
        mid,
        icon=DivIcon(
            icon_size=(120, 22),
            icon_anchor=(0, 0),
            html=f"<div style='font-size:12px;color:#d35400;background:white;padding:2px 4px;border-radius:3px;'>{d_km:.2f} km</div>"
        )
    ).add_to(site_map)

# 6) Render (make sure your notebook is Trusted)
site_map


In [35]:
import folium
from folium.features import DivIcon
from math import radians, sin, cos, asin, sqrt

# --- Ensure site_map and site_coord exist ---
# site_coord should be [lat, lon] of your chosen launch site.
# Example (uncomment and edit if needed):
site_coord = [28.5618571, -80.577366]

# --- Distance (Haversine, km) ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0088
    p1, p2 = radians(lat1), radians(lat2)
    dphi  = p2 - p1
    dlmb  = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(p1)*cos(p2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

def _pair(v, name):
    if v is None:
        return None
    if isinstance(v, str):
        parts = [p.strip() for p in v.replace("|", ",").split(",")]
        if len(parts) != 2:
            raise ValueError(f"{name} must be [lat, lon], got {v!r}")
        v = [parts[0], parts[1]]
    return [float(v[0]), float(v[1])]

# Paste the coordinates you read with MousePosition (as [lat, lon]):
city_coord      = [28.538336, -81.379234]
railway_coord   = [28.572100, -80.649000]
highway_coord   = [28.572000, -80.585000]
coastline_coord = [28.563000, -80.567000]

palette = {"city":"#1b9e77", "railway":"#d95f02", "highway":"#7570b3", "coastline":"#e7298a"}

# Mark the launch site for reference (optional)
folium.Circle(site_coord, radius=1200, color="#2b8cbe", fill=True, fill_opacity=0.35).add_to(site_map)

for label, varname in [("city","city_coord"), ("railway","railway_coord"),
                       ("highway","highway_coord"), ("coastline","coastline_coord")]:
    coord = _pair(globals().get(varname), varname)
    if coord is None:
        continue

    # distance
    d_km = calculate_distance(site_coord[0], site_coord[1], coord[0], coord[1])

    # line from site to feature
    folium.PolyLine([site_coord, coord],
                    color=palette.get(label, "#2b8cbe"),
                    weight=3, opacity=0.9,
                    tooltip=f"{label.title()}: {d_km:.2f} km").add_to(site_map)

     # small dot at the feature (optional, for visibility)
    folium.CircleMarker(coord, radius=5,
                        color=palette.get(label, "#2b8cbe"),
                        fill=True, fill_opacity=0.9).add_to(site_map)

    # distance label marker at the feature
    folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(160, 24),
            icon_anchor=(0, 0),
            html=f"<div style='font-size:12px;color:#d35400;background:white;padding:2px 4px;border-radius:3px;'>{label.title()}: {d_km:.2f} km</div>"
        )
    ).add_to(site_map)

# Show map (make sure notebook is Trusted)
site_map